# Time series of inflow to system.

In [6]:
# Library imports.
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
def impulse(a, L, dt):
    b = np.pi / 2
    t = np.arange(0, L, dt)
    return a * np.cos(np.pi / L * t + b)**2, t

P, t = impulse(3, 10, 0.1)

plt.plot(t, P)
plt.show()

In [10]:
# Generate a long time series of pulses.
dt = 1 # Seconds.
T = 1 # Simulation length.
T_sec = T * 86400 # T in seconds.
t = np.arange(0, T_sec, dt)

# Pulse parameters.
#==========================
# Number of pulses.
n_pulse = 100

# Magnitude bounds.
a_bounds = (0.1, 5)

# Impulse length bounds in seconds.
L_bounds = (60, 60 * 30)

# Distribution parameters for generating similar precipitation time series.
# =================================
# Standard deviation for magnitude.
sig_a = 0.1

# Standard deviation for impulse length. Seconds.
sig_L = 60 * 2

# Standard deviation for impulse start. Seconds.
sig_imp = 60 * 5

# Number of precipitation time series to generate.
n_precip = 5

# Precipitation array.
#=====================================
# Precipitation array.
P = np.zeros((len(t), n_precip))

for i in range(n_pulse):
    # Sample a from uniform.
    a = np.random.uniform(a_bounds[0], a_bounds[1])

    # Sample start time from uniform.
    impulse_start = np.random.choice(t, 1)[0]

    # Impulse length.
    L = np.random.uniform(L_bounds[0], L_bounds[1])

    for j in range(n_precip):
        # Sample individual precipitation time series from parameters.

        # Magnitude. Ensure it is between bounds.
        a_j = np.min([np.max([np.random.normal(a, sig_a), a_bounds[0]]), a_bounds[1]])

        # Impulse start. Ensure it lies between 0 and max(t).
        impulse_start_j = np.min([np.max([int(np.random.normal(impulse_start, sig_imp)), 0]), np.max(t)])

        # Impulse length. Ensure it is between bounds.
        L_j = np.min([np.max([np.random.normal(L, sig_L), L_bounds[0]]), L_bounds[1]])

        # Generate impulse.
        P_impulse, t_impulse = impulse(a_j, L_j, dt)

        # Add to precipitation array at correct start point.
        idx = np.arange(impulse_start_j, np.min([impulse_start_j + len(P_impulse), len(t)]))

        P[idx,j] = P[idx,j] + P_impulse[:len(idx)]

In [ ]:
# Plotting.
plt.plot(t, P)
plt.xlabel('Time (s)')
plt.ylabel('Input')
plt.show()


In [ ]:
# Save as SWMM-readable .dat files.

# Starting date time.
start_dt = datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
dts = [start_dt + timedelta(seconds=i * dt) for i in range(len(t))]

# Date strings.
dates = [datetime.strftime(d, '%d-%m-%Y') for d in dts]
times = [datetime.strftime(d, '%H:%M:%S') for d in dts]

# File path.
file_dir = Path(r"inflow_data")

# Loop through time series and save.
for i in range(P.shape[1]):
    # File name.
    fname = f"P_{i}.dat"

    # Format data into a data frame.
    df = pd.DataFrame({'date': dates, 'time': times, 'P': P[:,i]})

    print(df.dtypes)

    # Save the dataframe as a dat file
    df.to_csv(file_dir / fname, index=False, header=False, sep='\t')